In [1]:
import cv2
import torch
import time
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tqdm import tqdm

In [ ]:
class MultiDetector():
    def __init__(self,video_path):
        
        self.frames_path=video_path
        self.model_type="DPT_Hybrid"
        self.device=device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.midas = torch.hub.load("intel-isl/MiDaS", self.model_type)

        self.midas.to(self.device)
        self.midas.eval()
        
        self.midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
        self.transform = self.midas_transforms.dpt_transform

        self.model = YOLO('yolov8n-seg.pt')

    def pre_works(self):
        cap = cv2.VideoCapture(self.frames_path)
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        ret, prev_frame = cap.read()
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        return cap,prev_gray,fps,frame_width,frame_height
    
    def estimate_speed(self,prev_gray,gray,fps):
        #flow = cv2.optflow.calcOpticalFlowDenseRLOF(prev_gray, gray, None)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        d_pixels = np.mean(np.abs(flow))
        speed_kmh = (d_pixels * 0.05 * fps * 3.6) / 0.03  ##0.06 scale factor // ## 0.06 px başına kaç metre
        return speed_kmh
    
    def midas_pred(self,img):
        input_batch = self.transform(img).to(self.device)
        with torch.no_grad():
            prediction = self.midas(input_batch)
            prediction = torch.nn.functional.interpolate(
                prediction.unsqueeze(1),
                size=img.shape[:2],
                mode='bilinear',
                align_corners=False,
            ).squeeze()
        depth_map = prediction.cpu().numpy()
        depth_map_normalized = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min())
        depth_map = 1.0 - depth_map_normalized  # Derinliği ters çevir
        return depth_map
    
    def yolo_predict(self,img):
         results = self.model.predict(img, verbose=False, device=0)
         return results
    
    def find_vehicle(self,img):
        results=self.yolo_predict(img)
        depth_map=self.midas_pred(img)
        closest_vehicle=None
        min_distance=float("inf")
        for predictions in results:
            if predictions is None or predictions.boxes is None or predictions.masks is None:
                continue  # Hiç nesne bulunamazsa atla
        
            for bbox, masks in zip(predictions.boxes, predictions.masks):
                for scores, classes, bbox_coords in zip(bbox.conf, bbox.cls, bbox.xyxy):
                    label = predictions.names[int(classes)]
                    if label not in ["car", "truck", "bus"]:  
                        continue  # Sadece araçları al

                    xmin, ymin, xmax, ymax = map(int, bbox_coords)
                    depth_values_bbox = depth_map[ymin:ymax, xmin:xmax]
                    depth_value = np.median(depth_values_bbox)
                    
                    scale_factor = 5  # Ölçekleme faktörü (ayarlanabilir)
                    distance = depth_value * scale_factor

                    # En yakın aracı belirle
                    if distance < min_distance:
                       
                        min_distance = distance
                        closest_vehicle = (xmin, ymin, xmax, ymax, label, scores, masks)
        if closest_vehicle:
            xmin, ymin, xmax, ymax, label, scores, masks = closest_vehicle

            # Bounding box çiz
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

            # Derinlik bilgisini ekrana yaz
            text = f"{label}: {round(float(scores) * 100, 1)}% - {min_distance:.2f}m"
            cv2.putText(img, text, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

 
    def __call__(self, *args, **kwds):
        cap,prev_gray,FPS,frame_width,frame_height=self.pre_works()
        frameId = 0
        start_time = time.time()
        fps = str()
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        progress_bar = tqdm(total=total_frames, desc="Processing Frames", unit="frame")
       
        fps_current=20
        while cap.isOpened():
            frameId+=1
            t_ret, t_frame = cap.read()
            img = t_frame
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            
            self.find_vehicle(img)
            
            
            if frameId % 10 == 0:
                end_time = time.time()
                elapsed_time = end_time - start_time
                fps_current = 10 / elapsed_time
                fps = f'FPS: {fps_current:.2f}'
                start_time = time.time()
            speed_kmh=self.estimate_speed(prev_gray,gray,fps_current)
            cv2.putText(img, fps, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1, cv2.LINE_AA)
            cv2.putText(img, f"Speed: {speed_kmh:.2f} km/h", (-30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.imshow('Filtered Vehicle Detection', img)
            prev_gray = gray
            if cv2.waitKey(1) & 0xFF == ord('q'):
                 break

            progress_bar.update(1)
        progress_bar.close()
        cap.release()
    
        cv2.destroyAllWindows()
    

In [ ]:
MultiDetector=MultiDetector(r"speed-challenge-train.mp4")

In [ ]:
MultiDetector()